In [1]:
import os
import sys
import math

import numpy
import pandas

# Generalized matrix operations:

def __extractNodes(matrix):
    nodes = set()
    for colKey in matrix:
        nodes.add(colKey)
    for rowKey in matrix.T:
        nodes.add(rowKey)
    return nodes

def __makeSquare(matrix, keys, default=0.0):
    matrix = matrix.copy()
    
    def insertMissingColumns(matrix):
        for key in keys:
            if not key in matrix:
                matrix[key] = pandas.Series(default, index=matrix.index)
        return matrix

    matrix = insertMissingColumns(matrix) # insert missing columns
    matrix = insertMissingColumns(matrix.T).T # insert missing rows

    return matrix.fillna(default)

def __ensureRowsPositive(matrix):
    matrix = matrix.T
    for colKey in matrix:
        if matrix[colKey].sum() == 0.0:
            matrix[colKey] = pandas.Series(numpy.ones(len(matrix[colKey])), index=matrix.index)
    return matrix.T

def __normalizeRows(matrix):
    return matrix.div(matrix.sum(axis=1), axis=0)

def __euclideanNorm(series):
    return math.sqrt(series.dot(series))

# PageRank specific functionality:

def __startState(nodes):
    if len(nodes) == 0: raise ValueError("There must be at least one node.")
    startProb = 1.0 / float(len(nodes))
    return pandas.Series({node : startProb for node in nodes})

def __integrateRandomSurfer(nodes, transitionProbs, rsp):
    alpha = 1.0 / float(len(nodes)) * rsp
    return transitionProbs.copy().multiply(1.0 - rsp) + alpha

def powerIteration(transitionWeights, rsp=0.15, epsilon=0.00001, maxIterations=1000):
    # Clerical work:
    transitionWeights = pandas.DataFrame(transitionWeights)
    nodes = __extractNodes(transitionWeights)
    transitionWeights = __makeSquare(transitionWeights, nodes, default=0.0)
    transitionWeights = __ensureRowsPositive(transitionWeights)

    # Setup:
    state = __startState(nodes)
    transitionProbs = __normalizeRows(transitionWeights)
    transitionProbs = __integrateRandomSurfer(nodes, transitionProbs, rsp)
    
    # Power iteration:
    for iteration in range(maxIterations):
        oldState = state.copy()
        state = state.dot(transitionProbs)
        delta = state - oldState
        if __euclideanNorm(delta) < epsilon: break

    return state


zadatak6=[[0,1/2,0,1/2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,1/3,2/3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
          [0,0,0,0,0,1/5,2/5,2/5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
          [0,0,0,0,0,1/3,0,1/3,0,0,0,0,1/3,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0,1/2,1/2,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1/2,0,0,1/2,0,0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,1/4,0,0,0,0,3/4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,1/2,0,0,1/2,0,0,0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0,0,0,1/2,1/6,0,0,0,0,0,0,1/3,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,3/4,0,0,0,0,1/4,0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,2/3,0,0,0,0,1/3,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1/3,0,2/3,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1/4,3/4,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24,1/24],
          [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2/3,0,1/3,0]]

zadatak7=[[0,1/2,0,0,1/4,1/4,0,0,0,0],[1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10],
          [1/3,0,0,0,0,1/6,0,1/2,0,0],[1/2,0,0,0,1/2,0,0,0,0,0],[0,0,0,0,0,0,0,1,0,0],
          [0,0,0,1/4,1/2,0,0,0,0,1/4],[0,1/5,2/5,0,0,0,0,2/5,0,0],[0,1,0,0,0,0,0,0,0,0],
          [0,0,3/8,0,3/8,0,0,0,0,1/4],[1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10]]

test=powerIteration(zadatak6)
test1=powerIteration(zadatak7)
print('Zadatak 6\n'+ str(test))
print('Zadatak 7\n'+ str(test1))


Zadatak 6
0     0.012463
1     0.017760
2     0.017495
3     0.125363
4     0.012463
5     0.087464
6     0.016700
7     0.072717
8     0.058179
9     0.012463
10    0.037369
11    0.058604
12    0.050714
13    0.034016
14    0.019647
15    0.041376
16    0.024671
17    0.028442
18    0.012463
19    0.019525
20    0.019525
21    0.175426
22    0.032694
23    0.012463
dtype: float64
Zadatak 7
0    0.085466
1    0.249549
2    0.069296
3    0.056601
4    0.126955
5    0.069756
6    0.041778
7    0.193345
8    0.041778
9    0.065478
dtype: float64
